In [ ]:
# randomnisation des utilisateurs group A/B pour un test A/B

import numpy as np

import numpy as np

np.random.seed(42)

users = df_merged['visitorid'].unique()

ab_groups = pd.DataFrame({
    'visitorid': users,
    'group': np.random.choice(['A', 'B'], size=len(users))
})

df_merged = df_merged.merge(ab_groups, on='visitorid')

